This notebook has the code to implement the feature sign search algorithm. The reference paper used :  

H. Lee, A. Battle, R. Raina, and A. Y. Ng. "Efficient sparse coding algorithms". Advances in Neural Information Processing Systems 19, 2007.

This is the function for the algorithm in the paper

In [82]:
def FeatureSignSearch(A, y, gamma):
    ATA = n.dot(A.T, A)
    ATY = n.dot(A.T, y)
    YTY = n.dot(y.T, y)
    derivative = -2 * ATY
    threshold = 1e-15
    x= n.zeros(ATA.shape[0])
    #Creating theta vector 
    theta = n.zeros(ATA.shape[0], dtype = n.int8)
    activeSet = set()
    #This Line
    condition2 = n.inf    
    condition1 = 0    
    
    while condition2 > gamma or not n.allclose(condition1, 0):
        #Optimality condition (a) as per the paper satisfied
        if n.allclose(condition1, 0):
            #Find the max i
            i = n.argmax(n.abs(derivative) * (theta == 0))
            if derivative[i] > gamma:
                theta[i] = -1
                x[i] = 0
                activeSet.add(i)
            elif derivative[i] < -gamma:
                theta[i] = 1
                x[i] = 0
                activeSet.add(i)
            #if no modification in my active set and the optimality condition too is satisfied then terminate
            if len(activeSet) == 0:
                break
        
        activeSetIndices = n.array(sorted(activeSet))
        #
        AHTAH = ATA[n.ix_(activeSetIndices, activeSetIndices)]
        AHTYH = ATY[activeSetIndices]
        thetaH = theta[activeSetIndices]
        #safeguard for matrix to go singular (footnote 3) 
        
        temp1 = n.linalg.inv(AHTAH)
        temp2 = AHTYH - gamma * thetaH / 2        
        xnew = n.dot(temp1, temp2)
        thetanew = n.sign(xnew)
        xold = x[activeSetIndices]
        thetaFlip = n.where(abs(thetanew - thetaH) > 1)[0]
        
        if len(thetaFlip) > 0:
            optCurr = xnew
            optObj = (YTY + n.dot(xnew, n.dot(AHTAH, xnew)) - 2*n.dot(xnew, AHTYH) + gamma * abs(xnew).sum())
            
            for index in thetaFlip:
                a = xnew[index]
                b = xold[index]
                slope = b / (b-a)
                current = xold - slope * (xold - xnew)
                cost = YTY + (n.dot(current, n.dot(AHTAH, current)) - 2 * n.dot(current, AHTYH) + gamma * abs(current).sum())
                if cost < optObj:
                    optObj = cost
                    optSlope = slope
                    optCurr = current
        else:
            optCurr = xnew
        x[activeSetIndices] = optCurr
        zeroindices = activeSetIndices[n.abs(x[activeSetIndices]) < threshold]
        x[zeroindices] = n.int8(n.sign(x[activeSetIndices]))
        activeSet.difference_update(zeroindices)
        derivative = -2 * ATY + 2 * n.dot(ATA, x)
        z_opt = n.max(abs(derivative[theta==0]))
        nz_opt = n.max(abs(derivative[theta!=0] + gamma*theta[theta != 0]))
    return x

Reading the data from the dataset

In [83]:
#Python Imports
import numpy as n

A = n.array([[1,2],[4,5],[7,8]])
y = n.array([1,2,3])
gamma = 0.25
FeatureSignSearch(A, y, gamma)

array([ 0.        ,  0.38575269])